### Execução do algoritmo Apriori - Biblioteca mlxtend
http://rasbt.github.io/mlxtend/

# Bibliotecas

In [399]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from decimal import Decimal

# Carregamento dos Dados - .csv

In [400]:
anoArq = '2018'

In [401]:
# endereco = '../BaseDadosSelecionados/'
# endereco2 = 'Agrupado_Ocorrencia'

In [402]:
nomeArq = 'ES_datatran' + anoArq +'_Org_Apriori'

In [403]:
endereco= 'PorAno/'
# endereco= 'PorData/'

In [404]:
data = pd.read_csv(endereco + nomeArq + '.csv', encoding='ISO-8859-1', sep=';', header=None)

In [405]:
data.shape

(2640, 12)

In [406]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11
0,dia_semana=Segunda-feira,br=262.0,municipio=CONCEICAO DO CASTELO,causa_acidente=Ultrapassagem Indevida,tipo_acidente=Colisão transversal,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Crescente,condicao_metereologica=Nublado,tipo_pista=Simples,tracado_via=Reta,uso_solo=Não
1,dia_semana=Segunda-feira,br=101.0,municipio=JOAO NEIVA,causa_acidente=Falta de Atenção à Condução,tipo_acidente=Colisão traseira,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Decrescente,condicao_metereologica=Nublado,tipo_pista=Simples,tracado_via=Reta,uso_solo=Sim
2,dia_semana=Segunda-feira,br=101.0,municipio=VILA VELHA,causa_acidente=Velocidade Incompatível,tipo_acidente=Saída de leito carroçável,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Crescente,condicao_metereologica=Ignorado,tipo_pista=Simples,tracado_via=Não Informado,uso_solo=Não
3,dia_semana=Segunda-feira,br=101.0,municipio=ATILIO VIVACQUA,causa_acidente=Fenômenos da Natureza,tipo_acidente=Queda de ocupante de veículo,classificacao_acidente=Com Vítimas Feridas,fase_dia=Pleno dia,sentido_via=Crescente,condicao_metereologica=Chuva,tipo_pista=Simples,tracado_via=Reta,uso_solo=Não
4,dia_semana=Terça-feira,br=101.0,municipio=SERRA,causa_acidente=Falta de Atenção do Pedestre,tipo_acidente=Atropelamento de Pedestre,classificacao_acidente=Com Vítimas Feridas,fase_dia=Plena Noite,sentido_via=Decrescente,condicao_metereologica=Céu Claro,tipo_pista=Dupla,tracado_via=Reta,uso_solo=Sim


# Carregamento dos dados em uma matriz (Importante)

In [407]:
%%time
# data.shape[0] LINHAS e data.shape[1] COLUNAS

dataset = []
for i in range(0, data.shape[0]):
    dataset.append([str(data.values[i,j]) for j in range(0, data.shape[1])])

Wall time: 914 ms


In [408]:
# dataset

# Tratamento para a execução do algoritmo Apriori

http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/

In [409]:
%%time
te = TransactionEncoder()

Wall time: 0 ns


## 1º Transformação - Representação para uma matriz de boleanos

In [410]:
%%time
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

Wall time: 38 ms


In [411]:
# df

## 2ª Transformação - Representação esparsa
Exemplo 3 - Trabalhando com representações esparsas - http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/ 

In [412]:
# %%time
# oht_ary = te.fit(dataset).transform(dataset, sparse=True)
# sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)

In [413]:
# sparse_df

# Execução do algoritmo apriori

In [414]:
numMinSupport = 0.1

In [415]:
%%time
# sparse_df
frequent_itemsets = apriori(df, min_support=numMinSupport, use_colnames=True)

Wall time: 162 ms


In [416]:
# frequent_itemsets

In [417]:
# metric = lift

In [418]:
%%time
association_rules_saida = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
# association_rules_saida = association_rules(frequent_itemsets)

Wall time: 34 ms


In [419]:
saidaRecords = pd.DataFrame(association_rules_saida)

## Método para contabilizaro número de consequentes e antecedentes em cada regra

In [420]:
# Cria duas novas colunas contabilizando os antecedentes e consequents
saidaRecords["antecedent_len"] = saidaRecords["antecedents"].apply(lambda x: len(x))
saidaRecords["consequent_len"] = saidaRecords["consequents"].apply(lambda x: len(x))

## Filtro

In [421]:
saidaRecords = saidaRecords[ (saidaRecords['antecedent_len'] >= 2 ) & (saidaRecords['consequent_len'] >= 1) ] # & (saidaRecords['support'] >= 0.5) 

In [422]:
# saidaRecords = saidaRecords[(saidaRecords['support'] >= 0.6)]

In [423]:
saidaRecords.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
34,"(br=101.0, causa_acidente=Falta de Atenção à C...",(classificacao_acidente=Com Vítimas Feridas),0.315152,0.831818,0.276136,0.876202,1.053358,0.013988,1.358517,2,1
35,"(br=101.0, condicao_metereologica=Céu Claro)",(classificacao_acidente=Com Vítimas Feridas),0.405682,0.831818,0.339394,0.836601,1.005750,0.001940,1.029273,2,1
36,"(br=101.0, condicao_metereologica=Nublado)",(classificacao_acidente=Com Vítimas Feridas),0.144318,0.831818,0.118182,0.818898,0.984467,-0.001865,0.928656,2,1
37,"(br=101.0, fase_dia=Pleno dia)",(classificacao_acidente=Com Vítimas Feridas),0.392803,0.831818,0.338258,0.861138,1.035248,0.011517,1.211143,2,1
38,"(municipio=SERRA, br=101.0)",(classificacao_acidente=Com Vítimas Feridas),0.215530,0.831818,0.185606,0.861160,1.035274,0.006324,1.211335,2,1


In [424]:
saidaRecords.max()

antecedents           (uso_solo=Sim, br=101.0, sentido_via=Crescente...
consequents                (classificacao_acidente=Com Vítimas Feridas)
antecedent support                                             0.439773
consequent support                                             0.831818
support                                                        0.364773
confidence                                                            1
lift                                                            1.97475
leverage                                                       0.100919
conviction                                                          inf
antecedent_len                                                        4
consequent_len                                                        2
dtype: object

## Salvando os resultados do algoritmo em um .csv

In [425]:
saidaRecords.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
34,"(br=101.0, causa_acidente=Falta de Atenção à C...",(classificacao_acidente=Com Vítimas Feridas),0.315152,0.831818,0.276136,0.876202,1.053358,0.013988,1.358517,2,1
35,"(br=101.0, condicao_metereologica=Céu Claro)",(classificacao_acidente=Com Vítimas Feridas),0.405682,0.831818,0.339394,0.836601,1.005750,0.001940,1.029273,2,1
36,"(br=101.0, condicao_metereologica=Nublado)",(classificacao_acidente=Com Vítimas Feridas),0.144318,0.831818,0.118182,0.818898,0.984467,-0.001865,0.928656,2,1
37,"(br=101.0, fase_dia=Pleno dia)",(classificacao_acidente=Com Vítimas Feridas),0.392803,0.831818,0.338258,0.861138,1.035248,0.011517,1.211143,2,1
38,"(municipio=SERRA, br=101.0)",(classificacao_acidente=Com Vítimas Feridas),0.215530,0.831818,0.185606,0.861160,1.035274,0.006324,1.211335,2,1


## Substituição de '.' para ','

In [426]:
listCol = ['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction']
for i in listCol:
    saidaRecords[i] = saidaRecords[i].astype(str)
    saidaRecords[i] = saidaRecords[i].str.replace('.', ',')

## Ordena os dados baseado na confiança

In [427]:
# Ordena pela saidaRecords e salva em um novo DataFrame
saidaRecords = saidaRecords.sort_values(by='confidence', ascending=False)

## Salva os resultados em um arquivo .csv

In [428]:
# Salvando em um arquivo .csv
saidaRecords.to_csv(endereco + 'AprioriMlxtend_'+ 'ES_datatran' + anoArq + '.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

In [429]:
# Gambiarra para atualizar o arquivo com os dados ordenados, de acordo com a célula anterior
# saidaRecords = pd.read_csv(endereco + 'AprioriMlxtend_'+ 'ES_datatran' + anoArq + '.csv', encoding='ISO-8859-1', sep=';')

In [430]:
# Salvando em um arquivo .csv
# saidaRecords.to_csv(endereco + 'AprioriMlxtend_'+ 'ES_datatran' + anoArq + '.csv', encoding= 'ISO-8859-1', sep= ';', index=False)

### Filtro para facilitar a visualização dos resultados

In [431]:
# rules[ (rules['antecedent_len'] >= 2) &
#        (rules['confidence'] > 0.75) &
#        (rules['lift'] > 1.2) ]

In [432]:
# rules[rules['antecedents'] == {'Eggs', 'Kidney Beans'}]

## Filtros modelos para serem utilizados no apriori

In [433]:
# saidaRecords[ (saidaRecords['antecedent_len'] >= 3 ) &
#                    (saidaRecords['consequent_len'] >= 3) ]

In [434]:
# frequent_itemsets[ (frequent_itemsets['length'] == 2) &
#                    (frequent_itemsets['support'] >= 0.8) ]

In [435]:
# frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]

# ATIVIDADES FUTURAS

- Verificar se a coluna ['ignorados'] é necessária para analisar os resultados. <br>
Analisar o significado da coluna 'ignorados' no site da PRF. Ou no arquivo de descrição de tabelas.
***Resposta** : No link do anuário 2020, lá é apresentado alguns detalhes dos dados.

- Estudar os parâmetros do algoritmo apriori.

- Analisar e entende melhor a coluna 'pessoas'.

- Analisar a saída do 'lift' e 'conviction'.
***Resposta**: O problema está no momento que os dados 'lift' e 'conviction' são passados para o arquivo .csv.

No Jupyter os dados estão sendo apresentados da maneira correta.

# Print Final

In [436]:
print("TUDO EXECUTADO!")

TUDO EXECUTADO!
